# Cryptopunks: obtener datos de un cryptopunk

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import requests
from bs4 import BeautifulSoup

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.keys import Keys
from scrapy import Selector
import math

import mysql.connector
import random

import re

## Establecer conexión con la BD

In [2]:
# Connect to the new database
mydb = mysql.connector.connect(
  host="localhost",
  user="cecilia",
  password="cecilia",
  database="cryptopunks"
)

mycursor = mydb.cursor()



## Función: Obtener la información del cryptopunk

In [3]:
def obtener_cryptopunk(soup, mydb):
    # Obtener el tipo de cryptopunk
    h4 = soup.find_all('h4')
    tipo_cp = h4[0].text.split()[3]
    print(tipo_cp)
    
    # Obtener el nombre del owner, y saber si está a la venta y su precio
    varios =soup.find_all('div', class_="col-md-10 col-md-offset-1")

    for linea in varios:
        texto = linea.text
        # Busco la línea que comienza con ...
        if texto.strip().startswith('Current Market Status') and texto.find('currently')>0:
            # Extraigo el nombre del dueño
            if texto.find("This punk is currently owned by address")>0:
                owner = texto.split('address ')[1].split('.',1)[0]
                print('Owner:',owner)
        
            # Averiguo si está en venta
            precio_venta = 0
            if texto.find("This punk is currently for sale by owner")>0:
                sale = True
                precio_venta = texto.split('by owner for ')[1].split('Ξ',1)[0]
                print('Precio de venta', precio_venta)
            else:
                sale = False # Dice 'This punk has not been listed for sale by its owner'
            print('For sale',sale)
        
            # Averiguo si hay ofertas
            oferta = 0
            if texto.find("There is a bid of")>0:
                bids = True
                oferta = texto.split('is a bid of ')[1].split('Ξ',1)[0]
                print('Valor ofrecido', oferta)
             
            else:
                bids = False  # Dice "There are currently no bids on this punk."
            print('Bids:',bids)

    # Obtain the typeID
    mycursor.execute("SELECT id FROM ctypes WHERE name = %s", (tipo_cp.strip(),))
    id_tipo = mycursor.fetchone()[0]

    # Insert cryptopunk data into the table
    sql = "INSERT INTO cryptopunks (id, owner, id_type, for_sale, bids) VALUES (%s, %s, %s, %s, %s )"

    mycursor.execute(sql, (nro_cp, owner, id_tipo, sale, bids ))

    
    mydb.commit()        

## Función: Obtener la lista de transacciones realizadas

In [4]:
def obtener_transacciones(soup, mydb):
    
    def obtener_montos(str1):
        try:
            salida = re.search("(((\(\$)\s*\d+)([,]\d+){0,1}([,]\d+){0,1}([.]?\d{1,2})?)[MKB]?\)", str1)
            con_coma = salida[0][2:-1]
            sin_coma = con_coma.replace(',','')
            if sin_coma.find('M')>0:
                sin_coma = sin_coma[0:-1]
                monto_dols = float(sin_coma)
                monto_dols = monto_dols * 1000000
            elif sin_coma.find('K')>0:
                sin_coma = sin_coma[0:-1]
                monto_dols = float(sin_coma)
                monto_dols = monto_dols * 1000
            elif sin_coma.find('B')>0:
                sin_coma = sin_coma[0:-1]
                monto_dols = float(sin_coma)
                monto_dols = monto_dols * 1000000000
            else: 
                monto_dols = float(sin_coma)
            
            
            con_E = re.search("(\d+([,]\d+)?([.]\d{1,2})?)?K?\Ξ", str1)
            #print(con_E[0])

            con_E = con_E[0][0:-1].replace(',','')
            #print(con_E)
            
            if con_E.find('M')>0:
                con_E = con_E[0:-1]
                monto = float(con_E)
                monto = monto * 1000000
            elif con_E.find('K')>0:
                print('paso')
                con_E = con_E[0:-1]
                monto = float(con_E)
                monto = monto * 1000
            else:
                monto = float(con_E)
            return monto, monto_dols
        except:
            return 0,0
        
    text1 = soup.find_all('a', text = '')
    #print(text1)
    
    transacciones =[]
    
    transac =soup.find('table', class_="table")
    #print(transac)

    for tr in transac.find_all('tr'):
        lista = []
        for td in tr.find_all('td'):
            lista.append(td.text.strip())
        transacciones.append(lista)
    transacciones = transacciones[1:]
    print(transacciones)
    
  
    # Insert data into the table
    sql = "INSERT INTO transactions (id_cryptopunk, tr_type, tr_from, tr_to, amount, amount_dols, txn) VALUES (%s, %s, %s, %s, %s, %s, %s)"

    for tr in transacciones:
        # si la transaccion no esta vacia
        if len(tr)>0:
            # Hay un monto de transacción?
            monto = 0
            monto_dols = 0
            if len(tr[3])>0:
                monto, monto_dols = obtener_montos(tr[3])
            
            fecha = datetime.strptime(tr[4], "%b %d, %Y")
            
    
            mycursor.execute(sql, (nro_cp, tr[0],  tr[1], tr[2], monto, monto_dols, fecha))

    
    mydb.commit()

## Función: Obtener atributos del cryptopunk

In [5]:
def obtener_atributos(soup, mydb):
    cod =soup.find_all('div', class_="col-md-4")
    textos = []
    for c in cod:
        textos.append(c.find('a').text)
    
    atributos = textos[1:]
    print('Atributos:', atributos)
    
    # Insert data into the table attributes
    sql = "INSERT INTO cryptopunk_has_attributes (id_cryptopunk, id_attribute) VALUES (%s, %s)"
    

    for atr in atributos:
        # Obtain the typeID
        mycursor.execute("SELECT id FROM attributes WHERE name = %s", (atr.strip(),))
        id_atr = mycursor.fetchone()[0]
    
        mycursor.execute(sql, (nro_cp,id_atr))


    mydb.commit()

## Ejecución del proceso

In [6]:
# Rango de cryptopunks a scrapear
for nro_cp in range(9147,10000):
    
    # Define cryptopunk to scrap
    url = 'https://www.larvalabs.com/cryptopunks/details/'+str(nro_cp)

    # Use BeautifulSoup por webscrapping
    html=requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')

    obtener_cryptopunk(soup, mydb)
    obtener_transacciones(soup, mydb)
    obtener_atributos(soup, mydb)
    time.sleep(random.uniform(5.0, 8.0))

Female
Owner: 0x26f744
For sale False
Bids: False
[['Claimed', '', '0x26f744', '', 'Jun 23, 2017']]
Atributos: ['Mole', 'Horned Rim Glasses', 'Hot Lipstick', 'Blonde Bob']
Female
Owner: wilcox
For sale False
Bids: False
[['Transfer', 'Wilcox', 'wilcox.e…', '', 'Apr 25, 2021'], ['Transfer', '0x00d7c9', 'Wilcox', '', 'Sep 11, 2019'], ['Offered', '', '', '1Ξ ($306)', 'Jul 07, 2019'], ['Bid Withdrawn', 'Pixls', '', '0.40Ξ ($115)', 'Jul 06, 2019'], ['Bid', 'Pixls', '', '0.40Ξ ($116)', 'Jun 30, 2019'], ['Bid Withdrawn', 'Pixls', '', '0.50Ξ ($134)', 'Jun 16, 2019'], ['Offered', '', '', '4Ξ ($1,074)', 'Jun 16, 2019'], ['Bid', 'Pixls', '', '0.50Ξ ($134)', 'Jun 16, 2019'], ['Offered', '', '', '0.70Ξ ($118)', 'May 03, 2019'], ['Offered', '', '', '3Ξ ($3,151)', 'Jan 16, 2018'], ['Offered', '', '', '0.50Ξ ($680)', 'Jan 14, 2018'], ['Claimed', '', '0x00d7c9', '', 'Jun 23, 2017']]
Atributos: ['Medical Mask', 'Mohawk Dark']
Female
Owner: BlackBox…
For sale False
Bids: False
[['Sold', '0xa1179f', 'Blac

Male
Owner: 0x030def
For sale False
Bids: False
[['Transfer', '0xc272ed', '0x030def', '', 'Sep 12, 2021'], ['Offer Withdrawn', '', '', '', 'Apr 05, 2021'], ['Offered', '', '', '35Ξ ($64,049)', 'Mar 18, 2021'], ['Bid Withdrawn', '0x82ac0c', '', '15Ξ ($26,888)', 'Mar 16, 2021'], ['Offered', '', '', '37Ξ ($66,986)', 'Mar 16, 2021'], ['Bid', '0x82ac0c', '', '15Ξ ($26,677)', 'Mar 15, 2021'], ['Bid', '0x7760e0', '', '0.15Ξ ($40)', 'Jul 04, 2017'], ['Bid', '0x5b098b', '', '0.15Ξ ($40)', 'Jul 04, 2017'], ['Claimed', '', '0xc272ed', '', 'Jun 23, 2017']]
Atributos: ['Normal Beard Black', 'Cap Forward', 'Frown', 'Horned Rim Glasses']
Male
Owner: 9x9x9
For sale False
Bids: False
[['Sold', 'shrimpwe…', '9x9x9.et…', '102Ξ ($390,524)', 'Oct 16, 2021'], ['Offered', '', '', '102Ξ ($390,524)', 'Oct 16, 2021'], ['Offered', '', '', '107Ξ ($406,353)', 'Oct 15, 2021'], ['Offered', '', '', '288.88Ξ ($1.14M)', 'Sep 07, 2021'], ['Sold', '0xb4bb41', 'shrimpwe…', '113.99Ξ ($370,026)', 'Aug 28, 2021'], ['Offered'

Male
Owner: 0xb88f61
For sale False
Bids: False
[['Transfer', '0xca04e9', '0xb88f61', '', 'May 21, 2021'], ['(Unwrap)', 'WrappedC…', '0xca04e9', '', 'May 21, 2021'], ['(Wrap)', '0x6efd42', 'WrappedC…', '', 'May 21, 2021'], ['Transfer', 'illusive…', '0x6efd42', '', 'May 21, 2021'], ['Sold', 'daddykal…', 'illusive…', '19.88Ξ ($51,848)', 'May 20, 2021'], ['Bid Withdrawn', 'illusive…', '', '18.50Ξ ($48,249)', 'May 20, 2021'], ['Bid', 'illusive…', '', '18.50Ξ ($47,734)', 'May 19, 2021'], ['Bid Withdrawn', '0xb50ac7', '', '11.50Ξ ($46,663)', 'May 12, 2021'], ['Bid', '0xb50ac7', '', '11.50Ξ ($47,203)', 'May 12, 2021'], ['Offered', '', '', '19.88Ξ ($80,350)', 'May 11, 2021'], ['Sold', 'mabu.eth', 'daddykal…', '17Ξ ($68,710)', 'May 11, 2021'], ['Offered', '', '', '17Ξ ($68,710)', 'May 11, 2021'], ['Transfer', '0x48db19', 'mabu.eth', '', 'May 02, 2021'], ['Claimed', '', '0x48db19', '', 'Jun 23, 2017']]
Atributos: ['Bandana', 'Normal Beard', 'Regular Shades']
Male
Owner: 0x2ad822
For sale False
B

Male
Owner: 0x0e9aed
Precio de venta 149.99
For sale True
Bids: False
[['Offered', '', '', '149.99Ξ ($465,938)', 'Jan 19, 2022'], ['Offered', '', '', '179.99Ξ ($820,526)', 'Nov 14, 2021'], ['Offered', '', '', '240Ξ ($779,071)', 'Aug 28, 2021'], ['Offered', '', '', '199.99Ξ ($648,752)', 'Aug 28, 2021'], ['Offered', '', '', '179.99Ξ ($573,166)', 'Aug 24, 2021'], ['Offered', '', '', '159.99Ξ ($531,047)', 'Aug 23, 2021'], ['Offered', '', '', '129.99Ξ ($410,875)', 'Aug 19, 2021'], ['Offered', '', '', '139.99Ξ ($395,619)', 'Aug 05, 2021'], ['Bid Withdrawn', '0xae21e3', '', '65Ξ ($182,170)', 'Aug 05, 2021'], ['Offered', '', '', '119.99Ξ ($335,668)', 'Aug 05, 2021'], ['Sold', '0xb5e6ac', '0x0e9aed', '87Ξ ($229,819)', 'Aug 05, 2021'], ['Bid', '0xae21e3', '', '65Ξ ($176,921)', 'Aug 05, 2021'], ['Offered', '', '', '87Ξ ($221,089)', 'Aug 02, 2021'], ['Offered', '', '', '95Ξ ($249,106)', 'Aug 01, 2021'], ['Offer Withdrawn', '', '', '', 'Jul 31, 2021'], ['Offered', '', '', '67Ξ ($165,314)', 'Jul 31,

Male
Owner: punks
For sale False
Bids: False
[['Transfer', 'Pixls', 'punks.pi…', '', 'Dec 28, 2021'], ['Offer Withdrawn', '', '', '', 'Jun 18, 2021'], ['Offered', '', '', '100Ξ ($206,387)', 'Apr 08, 2021'], ['Offered', '', '', '66Ξ ($121,117)', 'Mar 09, 2021'], ['Offered', '', '', '900Ξ ($1.41M)', 'Mar 06, 2021'], ['Offer Withdrawn', '', '', '', 'Sep 28, 2020'], ['Offered', '', '', '50Ξ ($18,420)', 'Sep 15, 2020'], ['Offered', '', '', '20Ξ ($4,067)', 'May 22, 2020'], ['Offered', '', '', '5Ξ ($842)', 'Jan 17, 2020'], ['Offered', '', '', '3Ξ ($806)', 'Jun 16, 2019'], ['Offered', '', '', '1.50Ξ ($246)', 'May 05, 2019'], ['Sold', '0x00d7c9', 'Pixls', '0.35Ξ ($61)', 'Apr 20, 2019'], ['Bid', 'Pixls', '', '0.35Ξ ($61)', 'Apr 19, 2019'], ['Offered', '', '', '0.50Ξ ($87)', 'Apr 19, 2019'], ['Offered', '', '', '0.70Ξ ($122)', 'Apr 18, 2019'], ['Offered', '', '', '3Ξ ($3,151)', 'Jan 16, 2018'], ['Offered', '', '', '0.40Ξ ($544)', 'Jan 14, 2018'], ['Offered', '', '', '0.30Ξ ($94)', 'Aug 22, 2017']

...

Female
Owner: ddaavvee…
For sale False
Bids: False
[['Transfer', 'ddaavvee', 'ddaavvee…', '', 'Dec 18, 2021'], ['Offer Withdrawn', '', '', '', 'Aug 28, 2021'], ['Offered', '', '', '3.33KΞ ($9.73M)', 'Aug 06, 2021'], ['Offered', '', '', '777Ξ ($1.39M)', 'Mar 15, 2021'], ['Offer Withdrawn', '', '', '', 'Feb 24, 2021'], ['Offered', '', '', '259Ξ ($479,844)', 'Feb 12, 2021'], ['Offer Withdrawn', '', '', '', 'Feb 10, 2021'], ['Offered', '', '', '101Ξ ($40,564)', 'Nov 02, 2020'], ['Offered', '', '', '45Ξ ($17,034)', 'Sep 18, 2020'], ['Bid Withdrawn', '0x00bd3a', '', '0.40Ξ ($76)', 'Jul 11, 2017'], ['Bid', '0x00bd3a', '', '0.40Ξ ($98)', 'Jul 08, 2017'], ['Claimed', '', 'ddaavvee', '', 'Jun 23, 2017']]
Atributos: ['Black Lipstick', 'Spots', 'Horned Rim Glasses', 'Medical Mask', 'Straight Hair']
Female
Owner: 7572
Precio de venta 80
For sale True
Bids: False
[['Offered', '', '', '80Ξ ($327,384)', 'Dec 24, 2021'], ['Bid Withdrawn', '0xcac080', '', '65.20Ξ ($261,764)', 'Dec 12, 2021'], ['Sold', '

Female
Owner: BlackBox…
For sale False
Bids: False
[['Sold', '0x231707', 'BlackBox…', '2.77Ξ ($572)', 'May 27, 2020'], ['Offered', '', '', '2.77Ξ ($491)', 'Nov 18, 2018'], ['Offered', '', '', '0.72Ξ ($394)', 'Mar 23, 2018'], ['Offered', '', '', '0.75Ξ ($408)', 'Mar 23, 2018'], ['Offered', '', '', '0.35Ξ ($214)', 'Mar 15, 2018'], ['Claimed', '', '0x231707', '', 'Jun 23, 2017']]
Atributos: ['Eye Patch', 'Hot Lipstick', 'Wild Hair', 'Earring']
Female
Owner: 0x794098
For sale False
Bids: False
[['Offer Withdrawn', '', '', '', 'Jan 20, 2022'], ['Offered', '', '', '69Ξ ($262,821)', 'Jan 04, 2022'], ['Offered', '', '', '74Ξ ($280,832)', 'Jan 04, 2022'], ['Offer Withdrawn', '', '', '', 'Jan 04, 2022'], ['Offered', '', '', '86Ξ ($373,441)', 'Dec 06, 2021'], ['Bid Withdrawn', '0x9979d2', '', '60Ξ ($247,573)', 'Dec 06, 2021'], ['Bid', '0x9979d2', '', '60Ξ ($254,260)', 'Dec 04, 2021'], ['Offer Withdrawn', '', '', '', 'Dec 04, 2021'], ['Offered', '', '', '69.42Ξ ($286,562)', 'Dec 03, 2021'], ['Offe

Male
Owner: WrappedC…
For sale False
Bids: False
For sale False
Bids: False
[['(Wrap)', '0xda82fd', 'WrappedC…', '', 'Jul 25, 2021'], ['Transfer', '0xbdef23', '0xda82fd', '', 'Jul 25, 2021'], ['Transfer', 'sunfire.…', '0xbdef23', '', 'Jul 25, 2021'], ['Sold', '✨🚀🌙.et…', 'sunfire.…', '15.85Ξ ($31,422)', 'Apr 08, 2021'], ['Bid Withdrawn', 'Deeze', '', '14.50Ξ ($28,718)', 'Apr 07, 2021'], ['Bid', 'Deeze', '', '14.50Ξ ($28,718)', 'Apr 07, 2021'], ['Bid Withdrawn', 'wookie.e…', '', '14.50Ξ ($28,793)', 'Apr 07, 2021'], ['Bid', 'wookie.e…', '', '14.50Ξ ($28,793)', 'Apr 07, 2021'], ['Bid Withdrawn', 'wookie.e…', '', '14Ξ ($27,800)', 'Apr 07, 2021'], ['Bid', 'wookie.e…', '', '14Ξ ($28,154)', 'Apr 07, 2021'], ['Offered', '', '', '15.85Ξ ($33,518)', 'Apr 07, 2021'], ['Offered', '', '', '16Ξ ($34,114)', 'Apr 06, 2021'], ['Offered', '', '', '16.25Ξ ($33,363)', 'Apr 02, 2021'], ['Offered', '', '', '18Ξ ($33,216)', 'Mar 30, 2021'], ['Offered', '', '', '21.15Ξ ($34,230)', 'Mar 26, 2021'], ['Offered', 

Female
Owner: 0x023ef7
Precio de venta 175
For sale True
Bids: False
[['Offered', '', '', '175Ξ ($598,820)', 'Oct 05, 2021'], ['Sold', 'keyboard…', '0x023ef7', '135Ξ ($403,557)', 'Sep 30, 2021'], ['Offered', '', '0x023ef7', '135Ξ ($403,557)', 'Sep 30, 2021'], ['Offer Withdrawn', '', '', '', 'Sep 30, 2021'], ['Offered', '', '', '149.90Ξ ($445,819)', 'Sep 30, 2021'], ['Offered', '', '', '150Ξ ($440,719)', 'Sep 28, 2021'], ['Offered', '', '', '165Ξ ($481,731)', 'Sep 25, 2021'], ['Offer Withdrawn', '', '', '', 'Sep 25, 2021'], ['Offered', '', '', '150Ξ ($462,855)', 'Sep 20, 2021'], ['Offered', '', '', '172.88Ξ ($581,290)', 'Sep 19, 2021'], ['Offer Withdrawn', '', '', '', 'Sep 14, 2021'], ['Offered', '', '', '130Ξ ($423,576)', 'Sep 13, 2021'], ['Offered', '', '', '155Ξ ($516,145)', 'Sep 10, 2021'], ['Offer Withdrawn', '', '', '', 'Sep 03, 2021'], ['Offered', '', '', '159.99Ξ ($544,228)', 'Sep 01, 2021'], ['Offer Withdrawn', '', '', '', 'Aug 31, 2021'], ['Offered', '', '', '159Ξ ($508,352)',

Male
Owner: 0xa9bfa9
For sale False
Bids: False
[['Bid Withdrawn', '0xaaa09a', '', '0.10Ξ ($332)', 'Jan 15, 2022'], ['Bid', '0xaaa09a', '', '0.10Ξ ($331)', 'Jan 15, 2022'], ['Bid', '0xc9b410', '', '0Ξ ($40)', 'Dec 17, 2021'], ['Bid Withdrawn', '0xe27e6a', '', '1Ξ ($1,338)', 'Jan 20, 2021'], ['Bid', '0xe27e6a', '', '1Ξ ($1,422)', 'Jan 19, 2021'], ['Bid Withdrawn', '0xd8e806', '', '0.05Ξ ($15)', 'Aug 08, 2017'], ['Bid', '0xd8e806', '', '0.05Ξ ($11)', 'Aug 04, 2017'], ['Claimed', '', '0xa9bfa9', '', 'Jun 23, 2017']]
Atributos: ['Cigarette', 'Earring', 'Crazy Hair', 'Smile']
Zombie
Owner: marspunk…
For sale False
Bids: False
[['Bid Withdrawn', 'n0b0dy.e…', '', '0.09Ξ ($413)', 'Nov 08, 2021'], ['Bid', 'n0b0dy.e…', '', '0.09Ξ ($411)', 'Nov 08, 2021'], ['Transfer', '0x944d32', 'marspunk…', '', 'Sep 29, 2021'], ['Transfer', '0xe5fa91', '0x944d32', '', 'Aug 02, 2021'], ['Transfer', '0x279679', '0xe5fa91', '', 'Jul 19, 2021'], ['Offer Withdrawn', '', '', '', 'Jul 08, 2021'], ['Bid Withdrawn', '0